# COGS 108 - EDA Checkpoint

# Names

- Gabriel Aganon
- Ramy Al-Hosri
- Sean Ng
- Jan Do
- Hwi Yeo

<a id='research_question'></a>
# Research Question

How is the amount of content in a video game (as dictated by the ESRB) predictive of its popularity on Twitch before and during the COVID-19 pandemic?

# Setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
plt.rcParams['figure.figsize'] = (30,15)
import patsy
import statsmodels.api as sm

In [ ]:
# import Twitch Dataset
URL = 'https://raw.githubusercontent.com/ngseann/twitch-data/main/Twitch_game_data.csv'
twitch = pd.read_csv(URL)

In [ ]:
twitch

In [ ]:
# import ESRB Dataset
URL2 = 'https://raw.githubusercontent.com/ngseann/twitch-data/main/Video_games_esrb_rating.csv'
esrb = pd.read_csv(URL2)

In [ ]:
esrb

# Data Cleaning

Following criticisms of the first checkpoint, we decided to further clean our data. Initially, we removed any rows with empty columns. After that, we then chose a column to merge the 2 datasets, which contains the titles of various video games. Then, we standardized the names of such to include as many as we can upon merging the datasets.

With those intial steps done, we created 2 new columns. The 1st new column contains the sum of the binary values from the esrb dataset's columns. We term this column "num_content" and further explain what it means in the context of our project down below.

The 2nd new column contains strings. Such strings are combinations of values from the Year and Month columns. As such, it is not coincidence that the name of the column is "month-year". For example, one value from this column would be "01-2020". Given that we are examining a specific period in time, this column contains the "month-year" in which data is collected from January 2019 to September 2021.

Beyond the steps outlined above, the rest of our cleaning is nothing special. We simply removed columns that we deem irrelevant to our project. Moreover, we sorted columns according to time (the columns "Year" and "Month").

In [ ]:
twitch.isnull().sum()

In [ ]:
twitch_null = twitch[twitch.isnull().any(axis = 1)]
twitch_null

In [ ]:
twitch.shape

In [ ]:
twitch = twitch.dropna(subset = ['Game'])

In [ ]:
twitch.shape

In [ ]:
esrb.isnull().sum()

In [ ]:
esrb.shape

In [ ]:
# matching the name of the column for which we are gonna merge the datasets
esrb = esrb.rename(columns={'title':'Game'})

In [ ]:
# creating a function for standardizing game names
def standardize_game(string):
    string = string.lower()
    string = string.strip()
    
    # removing intellectual property symbols
    if '™' in string: 
        string = string.replace('™', '')
    elif '®' in string:
        string = string.replace('®', '')
    elif '©' in string:
        string = string.replace('©', '')
    
    # strip again
    string.strip()
    
    return string

In [ ]:
# standardizing Game columns of twitch and esrb datasets
twitch['Game'] = twitch['Game'].astype(str).apply(standardize_game)
esrb['Game'] = esrb['Game'].astype(str).apply(standardize_game)

In [ ]:
# merge the 2 datasets through the Game column
bigData = twitch.merge(esrb)
# we also picked the years of interest, which are from 2019 to 2021
bigData = bigData.loc[(bigData['Year'] >= 2019) & (bigData['Year'] <= 2021)]

In [ ]:
# we add a column num_content, which contains the sums of the ESRB columns
esrb_columns_to_add = bigData.iloc[:, 13:].sum(axis = 1)
bigData = bigData.assign(sum = esrb_columns_to_add)
bigData = bigData.rename(columns = {'sum':'num_content'})
bigData = bigData.reset_index()

The column "num_content" (a shorthand for number of content) takes the sum of the preceding columns that are derived from the ESRB dataset. In other words, it adds up how much content that a game contains that the ESRB deems as important to that game's identity (and thus why it would end up being rated E, T, M, etc). "num_content" is what we are referring to when we say "amount of content (dictated by the ESRB) in a video game".

In [ ]:
# keeping Game, Month, Year, Avg_viewers, and everything after console
bigData = bigData.drop(['index', 'Rank', 'Hours_watched',
       'Hours_Streamed', 'Peak_viewers', 'Peak_channels', 'Streamers', 
        'Avg_channels', 'Avg_viewer_ratio', 'console'], axis = 1)
# the columns above are irrelevant to what we are doing

In [ ]:
# putting data from month and year column into a single column month-year
# this columns exists so that doing line plots will not be difficult
bigData['month-year'] = bigData['Month'].astype('str') + '-' + bigData['Year'].astype('str')

In [ ]:
# sorting the data by the two columns we made: month-year and num_content
bigData = bigData.sort_values(by = ['Year', 'Month'], ascending = True).reset_index(drop = True)

In [ ]:
# show final table for our main dataset labeled bigData
bigData

In [ ]:
bigData.shape

# Data Analysis & Results (EDA)

Our EDA (and various analyses) are split into 4 sections. The first section delves into the video games and what we can infer about such. Afterwards, the following section checks whether the dependent variable of interest (Twitch viewership) is normally disributed or not. When the distribution is determined to be normal, the 3rd section analyzes num_content and Twitch viewership as a linear relationship, utilizing regression as a means for testing whether num_content has predictive power over Twitch viewership. Finally, after doing such, we examine if this predictive power holds true from January 2019 to September 2021 by examining trends of Twitch viewership divided according to num_content.

# 1: What games are we dealing with?

In [ ]:
# count of games per num_content
# group by game name and num_content to get unique games and their num_content
game_by_content = bigData.groupby(['Game','num_content']).count().reset_index()
# group by num_content to get count of unique games per num_content
game_by_content.groupby('num_content')['Game'].count()

In [ ]:
# count of games per esrb_rating
# group by game name and num content to get unique games and their num_content
game_by_esrb = bigData.groupby(['Game','esrb_rating']).count().reset_index()
# group by num_content to get count of unique games per num_content
game_by_esrb.groupby('esrb_rating')['Game'].count()

In total, we are dealing with 196 games. With num_content <= 5, we have a reasonable amount of games. However, with num_content = 6, we only have 2 games.

In [ ]:
# below are the outlier games in which num_content = 6
game_by_content[game_by_content['num_content'] == 6].reset_index(drop = True).iloc[:, :2]

In [ ]:
bigData[bigData['Game'] == 'danganronpa v3: killing harmony']

In [ ]:
bigData[bigData['Game'] == 'grand theft auto v']

Of the 2 games in which num_content = 6, there really only 1 game that appears from 2019 to 2021: grand theft auto v. By contrast, danganronpa v3 only appears once in 2020. Additionally, both games are rated M. However, does having a high num_content necessarily mean that a game is rated M?

In [ ]:
bigData[bigData['num_content'] == 1].reset_index(drop = True)

By looking at games with num_content = 1, we see that not all are rated the same. For examples, apex legends is rated T while a game like rocket league is rated E. This implies that esrb rating is not based on how many identifiable elements are in a game, but on what those identifiable elements are. Our analysis should consider this finding. 

# 2: What is the distribution of viewership within our dataset?

In [ ]:
#
sns.histplot(bigData['Avg_viewers'], bins = 40, color = 'g')

Overall, Twitch viewership of the games within our dataset (measured in terms of average viewers or "Avg_viewers") is not normally distributed in our dataset. This implies that while there are certainly outliers, the majority of games are not necessarily viewed frequently by people on Twitch. As such, the distribution above is right-skewed, implying that we must perform a logarithmic transformation with our data, which we do below.

In [ ]:
# applying a logarithm over Avg_viewers with an offset of 0.1
bigData['viewers_log10'] = np.log10(bigData['Avg_viewers'] + 0.1)
bigData

In [ ]:
# checking if viewers_log10 becomes a normal distribution when plotted, which it does
# however, there are outliers to the right
sns.histplot(bigData['viewers_log10'], bins = 40, color = 'g')

# 3: Linear Regression

In [ ]:
sns.lmplot (
    x = 'num_content',
    y = 'viewers_log10',
    data = bigData,
    fit_reg = True,
    scatter_kws = {'color' : 'red'},
    line_kws = {'color' : 'blue'}
)

In [ ]:
outcome, predictors = patsy.dmatrices('viewers_log10 ~ num_content', bigData)

mod_log = sm.OLS(outcome, predictors)
fit_res = mod_log.fit()
print(fit_res.summary())

Performing a linear regression with our independent variable being num_content and viewers_log10 (the Twitch viewership of our dataset that we performed a logarithmic transformation upon), we end with an effect size of -0.0410 (because Twitch viewership was logarithmically transformed, this is even lower). Based on our confidence interval (ranging from -0.065 and -0.017) and p = 0.025, we conclude that there is an inverse relationship between num_content and Twitch viewership: as num_content increases, Twtich viewership (and thus popularity) decreases. This runs counter to what we expected in which we expected a direct relationship. However, does this hold up before and during the COVID-19 pandemic? 

## Comparing before and after Covid

We mark March 2020 as the start of the COVID era, so anything before 2020 is the before and anything from January 2020 on is the after

In [ ]:
before = bigData[bigData['Year'] <= 2019]
after = bigData[bigData['Year'] > 2019]

## Before COVID

In [ ]:
sns.lmplot (
    x = 'num_content',
    y = 'viewers_log10',
    data = before,
    fit_reg = True,
    scatter_kws = {'color' : 'red'},
    line_kws = {'color' : 'blue'}
)

In [ ]:
outcome, predictors = patsy.dmatrices('viewers_log10 ~ num_content', before)

mod_log = sm.OLS(outcome, predictors)
fit_res = mod_log.fit()
print(fit_res.summary())

## After COVID

In [ ]:
sns.lmplot (
    x = 'num_content',
    y = 'viewers_log10',
    data = after,
    fit_reg = True,
    scatter_kws = {'color' : 'red'},
    line_kws = {'color' : 'blue'}
)

In [ ]:
outcome, predictors = patsy.dmatrices('viewers_log10 ~ num_content', after)

mod_log = sm.OLS(outcome, predictors)
fit_res = mod_log.fit()
print(fit_res.summary())

An interesting relationship we see here is that before and after COVID both have negative coefficients, and thus negative relationships, but in the case of after COVID times, there is a stronger correlation between num_contents and viewership.

# 4: Line Plots - Observing Trends Over Time

The lineplots we generate contain both lines (representing each level of "num_content") and their respective confidence intervals. While the lineplot for the overall viewership from January 2019 to September 2021 is simple enough to read, the lineplot for the viewership according to "num_content" is difficult to understand. So, for the purpose of readability, we also decided to generate the 2nd lineplot without the confidence interval. Moreover, we generated lineplots for each level of "num_content" (and their respective confidence intervals).

In [ ]:
# lineplot for overall Twitch (average) viewership from January 2019 to September 2021 
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = bigData
)

As we anticipated, Twitch viewership before the COVID-19 pandemic (the onset of which occurred on 3-2020) is far lower than during such. With more people, indoors, there we expected that more people would be on Twitch to view other playing video games. By dividing this line according to num_content, how does this trend hold up?

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    hue = 'num_content',
    data = bigData,
    # the color of the data is done in order (num_content = 1, num_content = 2, etc)
    palette = ['#ff0502','#7d7d7d','#b36a0c','#0c87b3','#6d874a','#5c78d8'],
    ci = None
)

We see that overall, the trend of more viewers coming to Twitch during the COVID-19 pandemic holds true for all levels of num_content. True to our linear regression test, we see that the most popular games over time have num_content = 1 (we must note that we do not count num_content = 6 as we have identified such to largely be outliers). However, for 1 < num_content < 6, we notice that the inverse relationship of the linear regression test does not necessarily hold up. For example, during the COVID-19 pandemic (3-2021 onwards), we would anticipate that following num_content = 1, games with num_content = 2 would be the next most popular on Twitch. However, we find that throughout the COVID-19 pandemic games that have a num_content with values 2, 3 and 5 shift in terms of being the 2nd most popular.

(Below we run lineplots for each level of num_content to further solidify that overall, viewership increases going from 2019 to 2021 in which the COVID-19 pandemic runs from 3-2020 onwards.)

In [ ]:
# these are subsets of bigData based on num_content
num_level1 = bigData[bigData['num_content'] == 1]
num_level2 = bigData[bigData['num_content'] == 2]
num_level3 = bigData[bigData['num_content'] == 3]
num_level4 = bigData[bigData['num_content'] == 4]
num_level5 = bigData[bigData['num_content'] == 5]
num_level6 = bigData[bigData['num_content'] == 6]

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level1
).set(title='Twitch Viewership with num_content = 1')

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level2
).set(title='Twitch Viewership with num_content = 2')

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level3
).set(title='Twitch Viewership with num_content = 3')

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level4
).set(title='Twitch Viewership with num_content = 4')

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level5
).set(title='Twitch Viewership with num_content = 5')

In [ ]:
sns.lineplot(
    x = 'month-year',
    y = 'viewers_log10',
    data = num_level6
).set(title='Twitch Viewership with num_content = 6')